In [0]:
orders_items=spark.sparkContext.textFile("dbfs:/FileStore/part_00000-5")
orders=spark.sparkContext.textFile("dbfs:/FileStore/orders_wh.csv")
customers=spark.sparkContext.textFile("dbfs:/FileStore/part_00000-1")

## customers data looks like this 
customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode

In [0]:
customers.take(5)

Out[2]: ['1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521',
 '2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126',
 '3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725',
 '4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069',
 '5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,"10 Crystal River Mall ",Caguas,PR,00725']

## orders data looks like this
order_id,order_date,order_customer_id,order_status

In [0]:
orders.take(5)

Out[3]: ['order_id,order_date,customer_id,order_status',
 '1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED']

## order_items data looks like this
order_item_id,order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price

In [0]:
orders_items.take(5)

Out[4]: ['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

# Find top 10 customers who have spent the most amount

In [0]:
orders_item_mapped=orders_items.map(lambda x: (x.split(",")[1],x.split(",")[4]))

In [0]:
orders_item_mapped.take(5)

Out[6]: [('1', '299.98'),
 ('2', '199.99'),
 ('2', '250.0'),
 ('2', '129.99'),
 ('4', '49.98')]

In [0]:
orders_mapped=orders.map(lambda x: (x.split(",")[0],x.split(",")[2]))

In [0]:
orders_mapped.take(5)

Out[8]: [('order_id', 'customer_id'),
 ('1', '11599'),
 ('2', '256'),
 ('3', '12111'),
 ('4', '8827')]

In [0]:
joined_rdd=orders_item_mapped.join(orders_mapped)

In [0]:
joined_rdd.take(5)

Out[10]: [('4', ('49.98', '8827')),
 ('4', ('299.95', '8827')),
 ('4', ('150.0', '8827')),
 ('4', ('199.92', '8827')),
 ('10', ('199.99', '5648'))]

In [0]:
joined_rdd_mapped=joined_rdd.map(lambda x: (x[1][1],float(x[1][0])))

In [0]:
joined_rdd_mapped.take(5)

Out[12]: [('8827', 49.98),
 ('8827', 299.95),
 ('8827', 150.0),
 ('8827', 199.92),
 ('5648', 199.99)]

In [0]:
final=joined_rdd_mapped.reduceByKey(lambda x,y:x+y)

In [0]:
final.take(5)

Out[14]: [('5225', 4429.38),
 ('5375', 2209.7400000000002),
 ('12345', 2443.6500000000005),
 ('6762', 849.9300000000001),
 ('1347', 4504.57)]

In [0]:
final=final.sortBy(lambda x: x[1],False)

In [0]:
final.take(10)

Out[16]: [('791', 10524.17),
 ('9371', 9299.03),
 ('8766', 9296.14),
 ('1657', 9223.71),
 ('2641', 9130.92),
 ('1288', 9019.11),
 ('3710', 9019.099999999999),
 ('4249', 8918.85),
 ('5654', 8904.95),
 ('5624', 8761.98)]

# Top 10 productid's with most quantities sold

In [0]:
orders_items_mapped=orders_items.map(lambda x: (x.split(",")[2],int(x.split(",")[3])))

In [0]:
orders_items_mapped.take(10)

Out[18]: [('957', 1),
 ('1073', 1),
 ('502', 5),
 ('403', 1),
 ('897', 2),
 ('365', 5),
 ('502', 3),
 ('1014', 4),
 ('957', 1),
 ('365', 5)]

In [0]:
final2=orders_items_mapped.reduceByKey(lambda x,y:x+y).sortBy(lambda x :x[1],False)

In [0]:
final2.take(10)

Out[20]: [('365', 73698),
 ('502', 62956),
 ('1014', 57803),
 ('191', 36680),
 ('627', 31735),
 ('403', 22246),
 ('1004', 17325),
 ('1073', 15500),
 ('957', 13729),
 ('977', 998)]

# How many customers are from Caguas city

In [0]:
customers.take(5)

Out[21]: ['1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521',
 '2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126',
 '3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725',
 '4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069',
 '5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,"10 Crystal River Mall ",Caguas,PR,00725']

In [0]:
customers_mapped=customers.filter(lambda x:x.split(",")[6] == "Caguas")

In [0]:
customers_mapped=customers_mapped.map(lambda x:x.split(",")[6])

In [0]:
customers_mapped.take(5)

Out[24]: ['Caguas', 'Caguas', 'Caguas', 'Caguas', 'Caguas']

In [0]:
customers_mapped.count()

Out[25]: 4584

# Top 3 states with maximum customers 

In [0]:
customers_mapped=customers.map(lambda x:(x.split(",")[-2],1))

In [0]:
customers_mapped.take(5)

Out[27]: [('TX', 1), ('CO', 1), ('PR', 1), ('CA', 1), ('PR', 1)]

In [0]:
customers_mapped.reduceByKey(lambda x,y:x+y).sortBy(lambda x: x[1],False).take(3)

Out[28]: [('PR', 4771), ('CA', 2012), ('NY', 775)]

# how many customers have spent more than $1000 in total

In [0]:
orders_mapped=orders.map(lambda x:(x.split(",")[0],x.split(",")[2]))

In [0]:
orders_mapped.take(5)

Out[95]: [('order_id', 'customer_id'),
 ('1', '11599'),
 ('2', '256'),
 ('3', '12111'),
 ('4', '8827')]

In [0]:
orders_item_mapped=orders_items.map(lambda x: (x.split(",")[1],float(x.split(",")[4])))

In [0]:
orders_item_mapped.take(5)

Out[101]: [('1', 299.98), ('2', 199.99), ('2', 250.0), ('2', 129.99), ('4', 49.98)]

In [0]:
over_1000=orders_item_mapped.join(orders_mapped)

In [0]:
over_1000.take(5)

Out[116]: [('4', (49.98, '8827')),
 ('4', (299.95, '8827')),
 ('4', (150.0, '8827')),
 ('4', (199.92, '8827')),
 ('10', (199.99, '5648'))]

In [0]:
over_1000_filtered=over_1000.filter(lambda x: x[1][0]>1000)

In [0]:
over_1000_filtered.take(5)

Out[118]: [('68837', (1999.99, '1223')),
 ('68875', (1999.99, '10637')),
 ('68736', (1999.99, '8462')),
 ('68778', (1999.99, '986')),
 ('68809', (1999.99, '5946'))]

In [0]:
over_1000_filtered.map(lambda x:x[1][1]).distinct().count()

Out[119]: 15